# Part 1

In [1]:
import pandas as pd
import numpy as np

In [4]:
with open('input.txt', 'r') as f:
    t0_str, busses_str = [line for line in f]

In [3]:
t0 = int(t0_str.strip())
t0

1000053

In [4]:
bus_ids = [int(bus_id) for bus_id in busses_str.strip().split(',') if bus_id != 'x']
bus_ids

[19, 37, 523, 13, 23, 29, 547, 41, 17]

In [5]:
bus_next_arrivals = []
for bus in bus_ids:
    arr = 0
    while arr < t0:
        arr+=bus
    bus_next_arrivals.append(arr)
bus_next_arrivals

[1000065,
 1000073,
 1000499,
 1000064,
 1000063,
 1000065,
 1000463,
 1000072,
 1000059]

In [6]:
df = pd.DataFrame({'bus_id':bus_ids,'arr':bus_next_arrivals})

In [7]:
df['wait'] = df['arr'] - t0

In [8]:
df.sort_values('wait', inplace=True)
df

,bus_id,arr,wait
8,17,1000059,6
4,23,1000063,10
3,13,1000064,11
0,19,1000065,12
5,29,1000065,12
7,41,1000072,19
1,37,1000073,20
6,547,1000463,410
2,523,1000499,446


In [9]:
df[['bus_id', 'wait']].iloc[0].prod()

102

# Part 2

For each bus_id, recode as `id - position` to get `id2`. let `increment=gcd(id2 for id2 in ID2)`. return increment multiple higher than highest bus iD except 1st, to which it must be equal to or greater than.

In [10]:
df['id2'] = df['bus_id'] - df.index

In [11]:
df['id2'].values

array([  9,  19,  10,  19,  24,  34,  36, 541, 521], dtype=int64)

In [12]:
np.gcd.reduce(df['id2'].values.ravel()), np.lcm.reduce(df['id2']) # 1, 32774797080
# submitted LCM (32774797080) as answer, too low
# just making sure I didn't have an off-by-one error: 32774797081 still too low

(1, 32774797080)

In [13]:
32774797080/19

1724989320.0

In [14]:
df.sort_index()

,bus_id,arr,wait,id2
0,19,1000065,12,19
1,37,1000073,20,36
2,523,1000499,446,521
3,13,1000064,11,10
4,23,1000063,10,19
5,29,1000065,12,24
6,547,1000463,410,541
7,41,1000072,19,34
8,17,1000059,6,9


In [15]:
df[['id2']].prod() # answer is too low! 269015534432640
# 269015534432640
# 32774797080... 4 orders of magnitude difference from prev attempt

id2    269015534432640
dtype: int64

In [16]:
# fuck it, I'd like to do this with numerical trickery, but I think it'll just be easier to do this computationally. 

#.... nah, that's both inelegant, and I think would take a really, really long time.

# It'll probably help to formalize what I'm looking for.

Find `t0` such that:

\begin{align}
t0 &= 19*\beta_0 \\
   &= 37*\beta_1 -1 \\  
   &= 523*\beta_2 - 2 \\
\ldots
   &= id_k * \beta_k - k
\end{align}

Alternatively:

$$
 \forall(k): 0 = id_k * \beta_k - k - t0 \\
 \beta_k > 0 \\
 \beta_k \in \mathbb{R}
$$

where the betas are constrained to be integer valued. We are solving for the $\beta_k$'s that minimize $t0$

In [17]:
#There's probably a better way to go about this, but right now I'm leaning towards integer programming.

In [18]:
import ortools
from ortools.sat.python import cp_model

In [19]:
import mip
# https://python-mip.readthedocs.io/en/latest/examples.html
from mip import Model, minimize

In [79]:
model = Model('mip-test7')

In [80]:
t0 = model.add_var(name="t0", var_type='i', lb=(df.bus_id - df.index).max()+1) # "i" - implicit constraint, can be removed
#t0 = model.add_var(name="t0", lb=(df.bus_id - df.index).max()) # "i" - implicit constraint, can be removed
model.objective = minimize(t0)
k = [v for v in df.index]
#b = [model.add_var(f"beta_{i}", var_type='i', lb=1) for i in k]
b = [model.add_var(f"beta_{i}", var_type='i', lb=1, obj=1) for i in k]

# Add positive constraints to beta_k's
#for b_k in b:
#    model += b_k >= 1
    
# add problem constraints
for ki, b_k, bus_id in zip(k, b, df.bus_id):
    #model += b_k * bus_id - ki - t0 == 0
    model += b_k * bus_id - ki == t0

In [ ]:
%%time 

## 11:28
#model.optimize(relax=True)
model.optimize()

In [76]:
model.num_solutions

1

In [77]:
model.objective_value

717.5114339846717

In [78]:
b[0].obj, b[0].x

(1.0, 32.352941176470594)

holy shit.... I'm pretty sure I've been using the wrong object this whole time. I don't actually have the correct offsets because I discarded all the x's when I read in the bus_ids. Ugh.

In [5]:
bus_offsets = [(int(bus_id), i) for i, bus_id in enumerate(busses_str.strip().split(',')) if bus_id != 'x']
bus_offsets
# ok, now we're cooking. Yeesh.

[(19, 0),
 (37, 13),
 (523, 19),
 (13, 37),
 (23, 42),
 (29, 48),
 (547, 50),
 (41, 60),
 (17, 67)]

New idea after playing with toy data in excel:

Product ~~at the top~~ of elements is the periodicity of the system. the question is really what the initial value is. which is affected by sort order.

## algorithm: 

- start with incrementer = to first position value
- increment up to first position where first two values are in phase.
- set incrementer to their product
- increment until third value is in phase.
- and so on

In [9]:
def phaser(bus_offsets):
    t0 = bus_offsets[0][0]
    increment = t0
    for i, (bus_id, offset) in enumerate(bus_offsets):
        if i==0:
            continue
        while True:
            t_bus = t0+offset
            if t_bus % bus_id == 0:
                increment *= bus_id
                break
            t0 += increment
    return t0

In [14]:
%%time
phaser(bus_offsets) # not just correct, but fast too!

Wall time: 0 ns


327300950120029